<h2>Steps summary</h2><br>
1) Scraping stock price from the desired quotes in a list<br>
2) Determine for each stock whether it is mean reversion or trend following<br>
3) Visulaize the result

<h2>1) Scraping stock price from the desired quotes in a list</h2><br>
copy quoute ของหุ้นใน SET50 จาก <a href="http://siamchart.com/stock/SET50">siamchart</a> แบบตรงๆ -> เอามาแปะลง excel -> copy จาก excel มาลง Jupyter ตาม cell ด้านล่าง

In [2]:
set50_str = '''ADVANC
AOT
BANPU
BBL
BDMS
BEAUTY
BEM
BGRIM
BH
BJC
BPP
BTS
CBG
CENTEL
CPALL
CPF
CPN
DELTA
DTAC
EA
EGCO
GLOBAL
GLOW
GPSC
HMPRO
INTUCH
IRPC
IVL
KBANK
KKP
KTB
KTC
LH
MINT
MTC
PTT
PTTEP
PTTGC
RATCH
ROBINS
SCB
SCC
SPRC
TCAP
TISCO
TMB
TOA
TOP
TRUE
TU'''

In [5]:
set50 = set50_str.split('\n')

In [26]:
print('total quote:',len(set50))

total quote: 50


ใช้ built-in API ของ YAHOO finance ใน pandas_datareader สำหรับดึงราคาหุ้น โดยฟอร์แมตในการดึงราคาหุ้นไทยจะอยู่ในฟอร์แมต QUOTE.BK เช่น ต้องการดึงราคาหุ้น ADVANC ก็ใช้ 'ADVANC.BK'

In [19]:
from pandas_datareader import data as pdr
from tqdm import tqdm,trange

In [12]:
stock_price = []
stock_name = []
for quote in tqdm(set50):
    try:
        stock_price.append(pdr.get_data_yahoo(f'{quote}.BK', start='2017-01-01', end='2018-11-30'))
        stock_name.append(quote)
    except:
        print("Error:", sys.exc_info()[0])
        print("Description:", sys.exc_info()[1])

100%|██████████| 50/50 [00:39<00:00,  1.22it/s]


<h2>2) Determine for each stock whether it is mean reversion or trend following</h2>

resample ข้อมูลราคาหุ้นรายวัน เป็นราคาปิดรายสัปดาห์(ตรงนี้แล้วแต่ แต่ละคนจะกำหนดช่วงเวลาว่าจะดูหุ้นเป็นรายวัน รายสัปดาห์ หรือรายเดือน) จากนั้นนำเอาราคาหุ้นมาคำนวณผลตอบแทนรายสัปดาห์ แล้วจึงคำนวณค่า autocorrelation (โดย default ถ้าไม่ได้กำหนดค่าอะไร จะเป็นการคำนวณค่าแบบ lag 1 คือ ค่า correlation ระหว่างๆ ผลตอบแทนธรรมดา กับ ผลตอบแทนที่เหลื่อมกันไป 1 หน่วยเวลา - สัปดาห์ ในที่นี้ )

In [20]:
auto_corr=[]
for i in trange(len(stock_price)):
    stock_price[i] = stock_price[i].resample(rule='W',how='last')
    stock_price[i]['Return'] = stock_price[i]['Adj Close'].pct_change()
    auto_corr.append(stock_price[i]['Return'].autocorr())

  0%|          | 0/50 [00:00<?, ?it/s]/Users/witchapong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 50/50 [00:00<00:00, 137.16it/s]


<h2>3) Visualize the result!</h2>

visualize ผลด้วย plotly... ด้านล่างนี้ผมใส่ ไม่ได้ใส่สีลงไปใน bar chart ใน code แต่สามารถดู chart แบบที่ใส่สีลงไปได้แล้ว<a href="https://plot.ly/dashboard/wdaroontham:42/view">ที่นี่</a> (ผมไป edit สีเอาบนเว็บของ plotly ครับ แหะๆ) ถ้าเอาเมาส์ไปจิ้มๆ ดูที่ bar chart จะเห็นว่า หุ้นที่มีลักษณะเป็น mean reversion หนักๆ ได้แก่ EGCO TCAP RATCH KBANK TMB และหุ้นที่มีลักษณะเป็น trend following หนักๆ ได้แก่ TU BEAUTY KTC EA และ KKP ครับบ

In [22]:
import pandas as pd
result_df = pd.DataFrame({'quote':stock_name,'auto_corr':auto_corr})

In [24]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Bar(
            x=result_df['auto_corr'].values,
            y=result_df['quote'].values,
            orientation = 'h'
)]

py.iplot(data, filename='SET50 autocorrelation')